In [1]:
import torch
import time
from torch.utils.data import DataLoader
from torch import nn
from transformers import AdamW, WEIGHTS_NAME, get_linear_schedule_with_warmup
import csv
import numpy as np
import os
import logging
from fp16 import FP16_Module, FP16_Optimizer
from parallel import DataParallelModel, DataParallelCriterion
from collections import OrderedDict
from utils.utils import *
from utils.settings import args, TASK_DICT, init_logging, MODEL_CONFIG, MODEL_CLASS, SPECIAL_TOKENS, CONFIG_CLASS
from utils.settings import TOKENIZER, SPECIAL_TOKEN_IDS, FILL_VAL, SAVE_NAME, FINAL_SAVE_NAME, TOKENS_WEIGHT, CONFIG_NAME
from scheduler import AnnealingLR
from regularizers import REG_TYPES, REG_TYPE_KEYS, Weight_Regularized_AdamW, Weight_Regularized_SGD
from torch.nn import CrossEntropyLoss
from transformers.adapters import ConfigUnion, AdapterConfig, PrefixTuningConfig
logger = logging.getLogger(__name__)

/home/student/2021/cs21mtech11006/anaconda3/envs/lamol/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/student/2021/cs21mtech11006/adapter-prompt/Adapter-Prompt/utils/settings.py:35: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  args = pd.Series()
No available GPUs!


NotImplementedError: No CPU mode available!

In [2]:
data_dir = "../../data"
sst = { "train":os.path.join(data_dir,"sst_to_squad-train-v2.0.json"),
           "eval":os.path.join(data_dir,"sst_to_squad-dev-v2.0.json"),
           "test":os.path.join(data_dir,"sst_to_squad-test-v2.0.json"),
      }

import pandas as pd
sst_train = pd.read_json(sst["train"])

sst_train["data"][0]

{'title': 'dummy',
 'paragraphs': [{'context': 'a stirring , funny and finally transporting re - imagining of beauty and the beast and 1930 s horror films',
   'qas': [{'id': '285b88e29d9911e984570c9d92873668',
     'answers': [{'answer_start': None, 'text': 'positive'}],
     'question': 'is this review negative or positive ?',
     'is_impossible': False}]}]}

In [3]:
if not args.debug:
    logging.getLogger("pytorch_transformers").setLevel(logging.WARNING)
    logging.getLogger("pytorch_transformers.tokenization_utils").setLevel(logging.CRITICAL)

make_dir(args.model_dir_root)

init_logging(os.path.join(args.model_dir_root, 'log_train.txt'))
logger.info('args = {}'.format(str(args)))

2023-04-12 16:53:19,161 - 0:19:59 - 0.0s - INFO - __main__ - args = Series([], dtype: object)


In [ ]:
task_id=0

In [ ]:
model = MODEL_CLASS.from_pretrained(args.model_name.cuda())
model.resize_token_embeddings(len(TOKENIZER))
adapter_name = args.tasks[task_id].replace(".", "_")
adapter_config = ConfigUnion(
                    AdapterConfig(mh_adapter=True, output_adapter=False, reduction_factor=3, non_linearity="relu"),
#                             AdapterConfig(mh_adapter=False, output_adapter=True, reduction_factor=8, non_linearity="relu"),
                    AdapterConfig(mh_adapter=False, output_adapter=True, reduction_factor=2, non_linearity="relu")
                )
#         adapter_config =  ConfigUnion(PrefixTuningConfig(bottleneck_size=800), ParallelConfig(),)
model.add_adapter(adapter_name, config=adapter_config)
model.set_active_adapters(adapter_name)
model = model.to(args.device_ids[0])
model.train_adapter(adapter_name)